In [1]:
import json
import nltk
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import SGD
import keras.backend as K
import pandas as pd
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [3]:
intents = json.loads(open('intents.json').read())
classes = []
documents = []
ignore_words = string.punctuation

In [4]:
from sklearn.preprocessing import OneHotEncoder

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #words.extend(pattern)
        #add documents in the corpus
        documents.append((pattern, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
df=pd.DataFrame(documents,columns=['question','tag'])

In [7]:
ohe = OneHotEncoder() 
tag_cat=df.tag

In [8]:
encoded_tag_cat= ohe.fit_transform(tag_cat.values.reshape(-1, 1)).toarray()

In [9]:
dummy=pd.DataFrame(encoded_tag_cat,columns=ohe.categories_)

In [10]:
df=pd.concat([df,dummy],axis=1)

In [11]:
df

question       tag  \
0                                        Hi there  greeting   
1                                     How are you  greeting   
2                                Is anyone there?  greeting   
3                                             Hey  greeting   
4                                            Hola  greeting   
..                                            ...       ...   
116             Where is Project head department?     cabin   
117                            where is cafeteria     cabin   
118                       guide me to the canteen     cabin   
119  How to improve team members domain knowledge    domain   
120    improving domain knowledge of team members    domain   

     (HR_related_problem,)  (Location,)  (Weather,)  (about,)  \
0                      0.0          0.0         0.0       0.0   
1                      0.0          0.0         0.0       0.0   
2                      0.0          0.0         0.0       0.0   
3                      0.0          0.0         0.0       0.0   
4                      0.0          0.0         0.0       0.0   
..                     ...          ...         ...       ...   
116                    0.0          0.0         0.0       0.0   
117                    0.0          0.0         0.0       0.0   
118                    0.0          0.0         0.0       0.0   
119                    0.0          0.0         0.0       0.0   
120                    0.0          0.0         0.0       0.0   

     (appointment status,)  (cabin,)  (check_leave,)  (commission,)  ...  \
0                      0.0       0.0             0.0            0.0  ...   
1                      0.0       0.0             0.0            0.0  ...   
2                      0.0       0.0             0.0            0.0  ...   
3                      0.0       0.0             0.0            0.0  ...   
4                      0.0       0.0             0.0            0.0  ...   
..                     ...       ...             ...            ...  ...   
116                    0.0       1.0             0.0            0.0  ...   
117                    0.0       1.0             0.0            0.0  ...   
118                    0.0       1.0             0.0            0.0  ...   
119                    0.0       0.0             0.0            0.0  ...   
120                    0.0       0.0             0.0            0.0  ...   

     (predict_delay,)  (predict_performance,)  (project_handling_queries,)  \
0                 0.0                     0.0                          0.0   
1                 0.0                     0.0                          0.0   
2                 0.0                     0.0                          0.0   
3                 0.0                     0.0                          0.0   
4                 0.0                     0.0                          0.0   
..                ...                     ...                          ...   
116               0.0                     0.0                          0.0   
117               0.0                     0.0                          0.0   
118               0.0                     0.0                          0.0   
119               0.0                     0.0                          0.0   
120               0.0                     0.0                          0.0   

     (search_department,)  (search_person_by_id,)  (solve_problems,)  \
0                     0.0                     0.0                0.0   
1                     0.0                     0.0                0.0   
2                     0.0                     0.0                0.0   
3                     0.0                     0.0                0.0   
4                     0.0                     0.0                0.0   
..                    ...                     ...                ...   
116                   0.0                     0.0                0.0   
117                   0.0                     0.0                0.0   
118  

In [12]:
#initiate Keras tokenizer and load text into internal vocabulary 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['question'])

In [13]:
#convert each question into numbers
X_train_que = tokenizer.texts_to_sequences(df['question'])

In [14]:
X_train_que[:10]

[[91, 51],
 [6, 12, 7],
 [1, 92, 51],
 [93],
 [94],
 [95],
 [96, 97],
 [52],
 [98, 7, 99],
 [100]]

In [15]:
# pad each sentence with zeros so each sentence has at least 15 numbers
X_train_que_padded = pad_sequences(X_train_que, 15)
X_train_que_padded[:5]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 91, 51],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6, 12,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 92, 51],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 93],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 94]])

In [16]:
Y_train=df.iloc[:,2:].values

In [17]:
Y_train[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(64, dropout=0, recurrent_dropout=0))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(Y_train[0]), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          7904      
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 43)                2795      
Total params: 39,691
Trainable params: 39,691
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
hist=model.fit(X_train_que_padded,Y_train,batch_size=64, epochs=110, verbose=1)

Epoch 1/110
2/2 [==============================] - 2s 12ms/step - loss: 3.7616 - accuracy: 0.0217
Epoch 2/110
2/2 [==============================] - 0s 11ms/step - loss: 3.7577 - accuracy: 0.0811
Epoch 3/110
2/2 [==============================] - 0s 12ms/step - loss: 3.7541 - accuracy: 0.1078
Epoch 4/110
2/2 [==============================] - 0s 11ms/step - loss: 3.7509 - accuracy: 0.1081
Epoch 5/110
2/2 [==============================] - 0s 13ms/step - loss: 3.7463 - accuracy: 0.1081
Epoch 6/110
2/2 [==============================] - 0s 12ms/step - loss: 3.7410 - accuracy: 0.0977
Epoch 7/110
2/2 [==============================] - 0s 10ms/step - loss: 3.7348 - accuracy: 0.1353
Epoch 8/110
2/2 [==============================] - 0s 11ms/step - loss: 3.7246 - accuracy: 0.1672
Epoch 9/110
2/2 [==============================] - 0s 10ms/step - loss: 3.7156 - accuracy: 0.1301
Epoch 10/110
2/2 [==============================] - 0s 11ms/step - loss: 3.7027 - accuracy: 0.1139
Epoch 11/110
2/2 [=

Epoch 83/110
2/2 [==============================] - 0s 11ms/step - loss: 0.7095 - accuracy: 0.8919
Epoch 84/110
2/2 [==============================] - 0s 11ms/step - loss: 0.6333 - accuracy: 0.9241
Epoch 85/110
2/2 [==============================] - 0s 10ms/step - loss: 0.6092 - accuracy: 0.9081
Epoch 86/110
2/2 [==============================] - 0s 11ms/step - loss: 0.5907 - accuracy: 0.9137
Epoch 87/110
2/2 [==============================] - 0s 10ms/step - loss: 0.5677 - accuracy: 0.9406
Epoch 88/110
2/2 [==============================] - 0s 11ms/step - loss: 0.5640 - accuracy: 0.9299
Epoch 89/110
2/2 [==============================] - 0s 11ms/step - loss: 0.5142 - accuracy: 0.9620
Epoch 90/110
2/2 [==============================] - 0s 10ms/step - loss: 0.5141 - accuracy: 0.9461
Epoch 91/110
2/2 [==============================] - 0s 10ms/step - loss: 0.4938 - accuracy: 0.9565
Epoch 92/110
2/2 [==============================] - 0s 11ms/step - loss: 0.4819 - accuracy: 0.9461
Epoch 93/1

In [22]:
model.save('RNN_model.h5', hist)

In [23]:
from keras.models import load_model
model = load_model('RNN_model.h5')

In [24]:
def bow(sentence):
    # tokenize the pattern    
    seq_sentence=tokenizer.texts_to_sequences([sentence])
    padded_sentence = pad_sequences(seq_sentence,15)
    return padded_sentence[0]

In [25]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence)
    res = model.predict(np.array([p]))[0]
    #print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": ohe.categories_[0][r[0]], "probability": str(r[1])})
    return return_list

In [31]:
#test model
predict_class('how have we improved our sale from last',model)

[{'intent': 'predict_performance', 'probability': '0.5112981'},
 {'intent': 'commission', 'probability': '0.28940946'}]

In [ ]:
ERROR_THRESHOLD = 0.25
results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD > 0.25]

In [ ]:
#Optional function

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision